In [6]:
import numpy as np # linear algebra
import json
from matplotlib import pyplot as plt
from skimage import color
from skimage.feature import hog
from sklearn import svm
from sklearn.metrics import classification_report,accuracy_score

from skimage.io import imread
from skimage.transform import resize
from skimage.feature import hog
from skimage import exposure
import matplotlib.pyplot as plt
import get_lodging_scores
import cv2  # importing cv
import imutils
import glob

#We need these in this file:
import load_read_name_extractor as lrne
import os
import pickle

In [7]:
def it_name_extract_labels_from_img(names):
    plots = []
    temp = []
    for i in range(len(names)):
        name_t = os.path.basename(names[i])
        #name_t = name_t.replace(".tif", "")
        for j in range(1,len(name_t)):
            if (names[i][-j] == "_"):
                temp.append(name_t[-(j-1):len(name_t)].replace(".jpeg", ""))
                break
        temp.append(name_t.replace("_"+ temp[0] + ".jpeg", ""))
        plots.append(temp)
        temp = []
    return plots

In [8]:
features_load = np.load('features/hog_features_64_128_hog_cells_8x8_block_2x2.npy', allow_pickle=True)
names_load = np.load('features/img_names_64_128_hog_cells_8x8_block_2x2.npy', allow_pickle=True)
print(len(features_load))
print(len(names_load))

4935
4935


In [9]:
labels = get_lodging_scores.get_labels(5)
#[ROI, Visual SCORE, FLIGHTFolder, BIN]
print(features_load[0])
ht_mean = features_load[0].mean(axis=0)
ht_mean

print(labels[0])

[0.20888097 0.16330611 0.04696022 ... 0.06301632 0.05749686 0.03150874]
(61, 50, '160607_Min', 2.0)


In [10]:
my_set = set(map(tuple, labels))
# convert set back to list of arrays
labels = list(map(np.array, my_set))

In [11]:
print(len(labels))

4936


In [12]:
names = it_name_extract_labels_from_img(names_load)

In [13]:
def match_pic_label(features, labels, names):
    n = len(labels)
    m = len(names)
    data_frame = []
    error = []
    for i in range(n):
        matched = False
        for j in range(m):
            if (str(labels[i][0]) == str(names[j][0]) and labels[i][2] == names[j][1]):
                data_frame.append(np.hstack((features[j], labels[i][3])))
                matched = True
        if (matched == False):
            error.append(np.hstack((labels[i][0], labels[i][2])))
    return data_frame, error

In [14]:
def match_pic_label_names(features, labels, names):
    
    m = len(labels)
    n = len(names)
    data_frame = []
    error = []
    for i in range(n):
        matched = False
        for j in range(m):
            if (str(labels[j][0]) == str(names[i][0]) and labels[j][2] == names[i][1]):
                data_frame.append(np.hstack((features[i], labels[j][3])))
                matched = True
        if (matched == False):
            error.append((names[i]))
    return data_frame, error

In [15]:
clf = svm.SVC(kernel="rbf", class_weight="balanced")
features_aar = np.array(features_load)
data_frame, error = match_pic_label(features_load, labels, names)
data_frame_names, error_names = match_pic_label_names(features_load, labels, names)
print(data_frame[0])
np.random.shuffle(data_frame)
data_frame = np.array(data_frame)


KeyboardInterrupt: 

In [ ]:
print(len(data_frame))
print(len(error))
print(len(data_frame_names))
print(len(error_names))

4915
21
4915
20


In [ ]:
print(error_names)

[['1', '170707_Min'], ['10', '170707_Min'], ['11', '170707_Min'], ['12', '170707_Min'], ['13', '170707_Min'], ['14', '170707_Min'], ['15', '170707_Min'], ['16', '170707_Min'], ['17', '170707_Min'], ['18', '170707_Min'], ['19', '170707_Min'], ['2', '170707_Min'], ['20', '170707_Min'], ['3', '170707_Min'], ['4', '170707_Min'], ['5', '170707_Min'], ['6', '170707_Min'], ['7', '170707_Min'], ['8', '170707_Min'], ['9', '170707_Min']]


In [ ]:
percentage = 80
partition = int(len(features_aar)*percentage/100)

In [ ]:
x_train, x_test = data_frame[:partition,:-1],  data_frame[partition:,:-1]
y_train, y_test = data_frame[:partition,-1:].ravel() , data_frame[partition:,-1:].ravel()
print(y_train)

clf.fit(x_train,y_train)

['1.0' '2.0' '1.0' ... '2.0' '1.0' '1.0']


SVC(class_weight='balanced')

In [ ]:
print(clf)

SVC(class_weight='balanced')


In [ ]:
y_pred = clf.predict(x_test)

In [ ]:
filename = 'SVM_linear_64_128_3bins_cells_8x8_block_2x2_balanced.sav'
pickle.dump(clf, open(filename, 'wb'))

In [ ]:
#loaded_model = pickle.load(open(filename, 'rb'))
#result = loaded_model.score(X_test, Y_test)

In [ ]:
print("Accuracy: "+str(accuracy_score(y_test, y_pred)))
print('\n')
print(classification_report(y_test, y_pred))

Accuracy: 0.7197518097207859


              precision    recall  f1-score   support

         1.0       0.85      0.83      0.84       684
         2.0       0.38      0.46      0.41       193
         3.0       0.59      0.46      0.52        90

    accuracy                           0.72       967
   macro avg       0.61      0.58      0.59       967
weighted avg       0.73      0.72      0.73       967



for i in range(len(names)):
    print(names[i])
for i in range(len(error)):
    print(error[i])
for i in range(len(names_load)):
    print(names_load[i])

In [ ]:
# Define cost matrix
num_classes = 20
cost_matrix = [[0]*num_classes for i in range(num_classes)]

for i in range(num_classes):
    for j in range(num_classes):
        cost_matrix[i][j] = abs(i-j)
cost_matrix

In [ ]:
def custom_scorer(y_true, y_pred):
    """
    Calculates the cost of classifying class i as class j.
    :param y_true: true labels
    :param y_pred: predicted labels
    :return: total cost
    """
    cost = 0
    for i in range(len(y_true)):
        cost += cost_matrix[int(y_true[i])][int(y_pred[i])]
    return cost

# Create scorer
scorer = make_scorer(custom_scorer, greater_is_better=False)

In [ ]:
# Define SVM model
svm = clf

# Define parameter grid for grid search
param_grid = {'C': np.logspace(-3, 3, 6)}

# Perform grid search using custom scorer
grid_search = GridSearchCV(svm, param_grid, scoring=scorer)
grid_search.fit(x_train,y_train)

# Print best parameters and score
print(grid_search.best_params_)
print(grid_search.best_score_)

In [ ]:
y_pred = grid_search.predict(x_test)
print("Accuracy: "+str(accuracy_score(y_test, y_pred)))
print('\n')
print(classification_report(y_test, y_pred))